# Análisis de sentimientos en el ámbito sociopolítico del Ecuador

## Importar módulos

In [ ]:
from core.login_twitter import login_twitter
from core.dirver_manager import get_driver, close_driver
from scraping.enlaces_scraper import guardar_enlaces_candidatos
from scraping.tweets_scraper import extraer_informacion_tweets
from scraping.comentarios_scraper import extraer_enlaces_comentarios_csvs
from scraping.contenido_comentarios_scraper import minar_texto_comentarios
from preprocessing.consolidar_corpus import consolidar_corpus_csvs
from preprocessing.limpieza_texto import limpiar_y_normalizar_corpus
from preprocessing.unificar_corpus import unificar_y_limpiar
from preprocessing.reestructurar_csv_a_json import reestructurar_csv_a_json_simple
from storage.mongodb_loader import cargar_json_a_mongo
from dotenv import load_dotenv
from pathlib import Path
import os

## Definición de las cadenas de consulta

In [2]:
candidatos_presidenciales_2025 = [
    #"Daniel Noboa", "Luisa González", "Wilson Gómez Vascones", "Jimmy Jairala",
    "Jorge Escala", "Andrea González Náder", "Henry Kronfle", "Carlos Rabascall", 
    # "Leonidas Iza", "Iván Saquicela", "Francesco Tabacchi", "Henry Cucalón",
    # "Enrique Gómez", "Víctor Araus", "Juan Iván Cueva", "Pedro Granja"
]


palabras_clave = [
    "elecciones 2025"
    # , "presidenciales 2025", "candidato presidencial", "candidatura 2025",
    # "campaña presidencial", "Ecuador 2025", "presidente Ecuador", "voto 2025",
    # "debate presidencial", "elecciones Ecuador"
]

## Configuración del driver

Al usar get_driver logramos que el web driver no se cierre lo que nos permitirá usar el mismo para realizar las acciones de web scraping

In [2]:
driver = get_driver()

## Llamadas a las variables de entorno

In [ ]:
# 1. Obtenemos el Directorio de Trabajo Actual (donde se ejecuta el Notebook)
#    Si ejecutas desde 'proyecto/webScraping', esto devolverá 'proyecto/webScraping'
current_dir = Path(os.getcwd())

# 2. Subimos un nivel para llegar a la carpeta 'proyecto/' (donde está .env)
#    Esto cambia la ruta de 'proyecto/webScraping' a 'proyecto'
base_dir = current_dir.parent

# 3. Construimos la ruta completa al archivo .env
dotenv_path = base_dir / '.env'

# --- Carga de Variables ---
load_dotenv(dotenv_path=dotenv_path)

# --- Llamada de Variables ---
username = os.getenv("X_USERNAME")
password = os.getenv("X_PASSWORD")
email = os.getenv("X_EMAIL")

## Inicio de sesión

Nos ayudará a mantener iniciada la sesión y evitar varios logeos

In [4]:
login_twitter(driver, username, password, email)

Paso 1: Ingresando Correo con simulación de tecleo...
Verificando si Twitter solicita usuario adicional...
Paso 4: Ingresando Contraseña con simulación de tecleo...
 - Inicio de sesión completado. Verificando redirección...


## Minar enlaces de las publicaciones en base a las cadenas de consulta

La cadena de consulta se compone de la unión de candidatos_presidenciales_2025 y palabras_claves
Por ejemplo: "Daniel Noboa elecciones 2025"
Una ve minados todos los enlaces a las publicaciones que aparecen en esta consulta, se guardarán los enlces en `Daniel_Noboa_enlaces_tweets.csv`, siendo "enlaces_tweets" el subijo para cada csv correspondiente a cada candidato

In [8]:
guardar_enlaces_candidatos(driver, candidatos_presidenciales_2025, palabras_clave)


Minando con query: Jorge Escala elecciones 2025
  Scroll #1...
  Scroll #2...
  Scroll #3...
  Scroll #4...
  Scroll #5...
  Scroll #6...
  Scroll #7...
  Scroll #8...
  Scroll #9...
  Scroll #10...
  Scroll #11...
  Scroll #12...
  Scroll #13...
  Scroll #14...
  Scroll #15...
  Scroll #16...
  Scroll #17...
  Scroll #18...
  Scroll #19...
  Scroll #20...
  Scroll #21...
  Scroll #22...
  Scroll #23...
  Scroll #24...
  Scroll #25...
Enlaces encontrados: 34
Guardado: datasets\raw\enlaces_publicaciones\Jorge_Escala_enlaces_tweets.csv
 - Pausa de 7.2 minutos antes del siguiente candidato...

Minando con query: Andrea González Náder elecciones 2025
  Scroll #1...
  Scroll #2...
  Scroll #3...
  Scroll #4...
  Scroll #5...
  Scroll #6...
  Scroll #7...
  Scroll #8...
  Scroll #9...
  Scroll #10...
  Scroll #11...
  Scroll #12...
  Scroll #13...
  Scroll #14...
  Scroll #15...
  Scroll #16...
  Scroll #17...
  Scroll #18...
  Scroll #19...
  Scroll #20...
  Scroll #21...
  Scroll #22...
 

## Minar contenido de cada publicación

In [10]:
extraer_informacion_tweets(driver, "datasets/raw/enlaces_publicaciones_4", "datasets/raw/informacion_publicaciones_4")

Guardado: datasets/raw/informacion_publicaciones_4\Luisa_Gonzalez_informacion_tweets.csv
Guardado: datasets/raw/informacion_publicaciones_4\Pedro_Granja_informacion_tweets.csv
Error en https://x.com/toanga/status/2005692220956107263/people: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x8a12d3
	0x8a1314
	0x68e6dd
	0x6d93a5
	0x6d977b
	0x720382
	0x6fb534
	0x71db13
	0x6fb2e6
	0x6cd321
	0x6ce1d4
	0xaf5314
	0xaf08cb
	0xb0d1aa
	0x8bb1d8
	0x8c31dd
	0x8a95d8
	0x8a9799
	0x893b28
	0x76835d49
	0x77c9d5db
	0x77c9d561

Error en https://x.com/OskarRaul77/status/2005121735616815405: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x8a12d3
	0x8a1314
	0x68e6dd
	0x6d93a5
	0x6d977b
	0x720382
	0x6fb534
	0x71db13
	0x6fb2e6
	0x6cd321
	0x6ce1d4
	0xaf5314
	0xaf08cb
	0xb0d1aa
	0x8bb1d8
	0x8c31dd
	0x8a95d8
	0x8a9799
	0x893b28
	0x76835d49
	0x77c9d5db
	0x77c9d561

Error en https://x.com/OskarRaul77/status/2005121735616815405: Message: 
Stacktrace:
Symbols

## Minar enlaces de los comentarios de cada publicación

In [7]:
extraer_enlaces_comentarios_csvs(driver, "datasets/raw/informacion_publicaciones_4", "datasets/raw/enlaces_comentarios_publicaciones_4", max_scrolls=100)


- Extrayendo comentarios de: https://x.com/ElPortalEcu/status/1969765620515324374
- Iniciando scroll de comentarios (max: 100)...
   Scroll 1: +3 nuevos comentarios - Total: 3
   Scroll 2: +10 nuevos comentarios - Total: 13
   Scroll 3: +16 nuevos comentarios - Total: 29
   Scroll 4: Sin nuevos comentarios (1/3) - Total: 29
   Scroll 5: Sin nuevos comentarios (2/3) - Total: 29
   Scroll 6: +2 nuevos comentarios - Total: 31
   Scroll 7: Sin nuevos comentarios (1/3) - Total: 31
   Scroll 8: Sin nuevos comentarios (2/3) - Total: 31
   Scroll 9: Sin nuevos comentarios (3/3) - Total: 31
- Fin de comentarios alcanzado en scroll 9
 ** Total de comentarios extraídos: 31 **

- Extracción completada: 30 comentarios
 - 30 comentarios en https://x.com/ElPortalEcu/status/1969765620515324374

- Extrayendo comentarios de: https://x.com/ElDatoEcua/status/1989337777075368238
- Iniciando scroll de comentarios (max: 100)...
   Scroll 1: +2 nuevos comentarios - Total: 2
   Scroll 2: +18 nuevos comentario

## Minar contenido comentarios de cada enlace

In [ ]:
minar_texto_comentarios(driver, "datasets/raw/enlaces_comentarios_publicaciones_1_original_A", "datasets/raw/datos_completos_publicacion", limite=None)


--- Procesando archivo: Andrea_Gonzalez_Nader_enlaces_comentarios.csv (121 comentarios/filas) ---
[1/121] Extraído: https://x.com/alperezs/status/1882100578...
[2/121] No se pudo extraer texto de: https://x.com/christianjhg33/status/1881... Error: TimeoutException
[3/121] Extraído: https://x.com/fabriciotc/status/18821785...
[4/121] Extraído: https://x.com/CaicedoFranc/status/188174...
[5/121] Extraído: https://x.com/SJhon_72/status/1881783465...
[6/121] No se pudo extraer texto de: https://x.com/ArturoHeart/status/2002441... Error: TimeoutException
[7/121] No se pudo extraer texto de: https://x.com/christianjhg33/status/1881... Error: TimeoutException
[8/121] Extraído: https://x.com/bolivar_lojan/status/18818...
[9/121] Extraído: https://x.com/FabianMore22837/status/188...
[10/121] Extraído: https://x.com/Daizmateo/status/188183264...
[11/121] Extraído: https://x.com/Daizmateo/status/188183264...
[12/121] Extraído: https://x.com/cesarnavegador/status/1881...
[13/121] Extraído: https:

## Cerrar driver

In [7]:
close_driver(driver)

## Unir CSV candidatos

In [8]:
consolidar_corpus_csvs("datasets/raw/datos_completos_publicacion", "datasets/raw/corpus_completo/corpus_comentarios_tweets.csv")

Creada carpeta de salida: datasets/raw/corpus_completo
Iniciando consolidación de CSVs en: datasets/raw/datos_completos_publicacion
Cargando: Andrea_Gonzalez_Nader_datos_completos.csv
Cargando: Carlos_Rabascall_datos_completos.csv
Cargando: Daniel_Noboa_datos_completos.csv
Cargando: Enrique_Gomez_datos_completos.csv
Cargando: Francesco_Tabacchi_datos_completos.csv
Cargando: Henry_Cucalon_datos_completos.csv
Cargando: Henry_Kronfle_datos_completos.csv
Cargando: Ivan_Saquicela_datos_completos.csv
Cargando: Jimmy_Jairala_datos_completos.csv
Cargando: Jorge_Escala_datos_completos.csv
Cargando: Juan_Ivan_Cueva_datos_completos.csv
Cargando: Leonidas_Iza_datos_completos.csv
Cargando: Luisa_Gonzalez_datos_completos.csv
Cargando: Pedro_Granja_datos_completos.csv
Cargando: Victor_Araus_datos_completos.csv
Cargando: Wilson_Gomez_Vascones_datos_completos.csv

Cargados 16 archivos. Concatenando...
Total de filas en el corpus: 19140
Corpus guardado en: datasets/raw/corpus_completo/corpus_comentarios

## Limpiar CSV

In [ ]:
limpiar_y_normalizar_corpus("datasets/raw/corpus_completo/corpus_comentarios_tweets.csv", "datasets/processed", nombre_archivo_salida="corpus_preprocesado_corregido.csv")

Archivo cargado. Filas iniciales: 19140
Filas nulas eliminadas. Filas restantes: 14633 (Eliminados: 4507)

Aplicando limpieza y normalización...

Eliminando duplicados...
Duplicados por texto_comentario_limpio eliminados. Filas restantes: 3228 (Eliminados: 11405)
Duplicados completos eliminados. Filas restantes: 3228 (Eliminados: 0)

Datos limpiados y guardados en 'datasets/processed\corpus_preprocesado.csv'


## Unificar Corpus Histórico y Limpieza final
En este paso, se toma el corpus procesado previamente ("viejo") y se une con los nuevos datos recolectados. Además, se aplica una limpieza profunda para eliminar duplicados finales antes de la conversión a JSON.

In [ ]:
# Este paso unifica el corpus histórico (webScraping) con el actual y elimina duplicados
# Genera: datasets/processed/corpus_preprocesado_unificado.csv
unificar_y_limpiar(
    ruta_original_str="datasets/processed/corpus_preprocesado.csv",
    ruta_nuevo_str="datasets/processed/corpus_preprocesado_corregido.csv",
    ruta_salida_final="datasets/processed/corpus_preprocesado_unificado.csv"
)

NameError: name 'unificar_y_limpiar' is not defined

## Convertir CSV a JSON

In [ ]:
reestructurar_csv_a_json_simple("datasets/processed/corpus_preprocesado_unificado.csv", "datasets/processed/corpus_completo.json")

Archivo CSV cargado: 4459 filas.
Reestructurando datos...

Archivo reestructurado guardado como 'datasets/processed/corpus_corregido.json'
Total de publicaciones (posts) únicas reestructuradas: 657


## Carga de JSON a MongoDB

In [2]:
print("--- Iniciando carga a Base de Datos ---")
cargar_json_a_mongo(
    ruta_json="datasets/processed/corpus_completo.json"
)

--- Iniciando carga a Base de Datos ---
- Conectado a MongoDB: tesis_politica_ecuador.corpus_politico
- Archivo leído. Preparando para procesar 657 documentos...
   - Procesados 100/657...
   - Procesados 200/657...
   - Procesados 300/657...
   - Procesados 400/657...
   - Procesados 500/657...
   - Procesados 600/657...

Proceso completado en MongoDB.
 - Documentos nuevos: 657
 - Documentos actualizados: 0
 - Total en colección: 657
